In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# read dataset
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cl_thyroid_conditions.csv')

# suffle data
cl_data = np.array(data)
m, n = cl_data.shape
np.random.shuffle(cl_data)                          # shuffle data

# split data for traning and testing
percentage_split = int(m*80/100)                    # percentage split: train, test= (80, 20)
train_data = cl_data[:percentage_split].T
test_data = cl_data[percentage_split:].T

# assign input and output
m_train, n_train = train_data.shape
X_train = train_data[0:n-1].T                       # set input train data
Y_train = np.expand_dims(train_data[-1], axis=-1)   # set output train data

m_test, n_test = test_data.shape
X_test = test_data[0:n-1].T                         # set input test data
Y_test = np.expand_dims(test_data[-1], axis=-1)     # set output test data

# activation function
# sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# derivative of sigmoid
def sigmoid_deriv(x):
    return x * (1 - x)

# ReLU function
def relu(x):
    return np.where(x > 0, x, 0)

# derivative of ReLU
def relu_deriv(x):
    return np.where(x > 0, 1, 0)

# mean squared error formula
def mean_squared_error(y_true, y_pred):
    x = np.square(y_true - y_pred)
    return (1/y_true.size) * np.sum(x)

# prediction formula
def get_predictions(predicted, t):
    return np.where(predicted > t, 1, 0)

# accuracy formula
def get_accuracy(y_pred, y_train):
    x = np.sum(y_pred == y_train) / y_train.size
    return x * 100

def init_param(X_train, Y_train):
    X   = np.array(X_train)                     # input
    Y   = np.array(Y_train)                     # output
    W1  = np.random.rand(m_train-1, 10) - 0.5   # 10 nodes in hidden layer
    W2  = np.random.rand(10, 10) - 0.5          # 10 nodes in hidden layer
    W3  = np.random.rand(10, 1) - 0.5           # 10 nodes in output layer
    B1  = np.zeros(shape=(1, 10))               # bias hidden layer 1
    B2  = np.zeros(shape=(1, 10))               # bias hidden layer 2
    B3  = np.zeros(shape=(1, 1))                # bias output layer
    return X, Y, W1, W2, W3, B1, B2, B3

In [39]:
from pandas._libs.tslibs.dtypes import Resolution
def feedforward(W1, W2, W3, B1, B2, B3, X):
    # layer 1
    Z1_in = X.dot(W1) + B1
    Z1 = relu(Z1_in)
    # layer 2
    Z2_in = Z1.dot(W2) + B2
    Z2 = relu(Z2_in)
    # layer 3
    Z3_in = Z2.dot(W3) + B3
    Z3 = sigmoid(Z3_in)
    return Z1, Z2, Z3

def backprop(Z1, Z2, Z3, W1, W2, W3, X, Y):
    # layer 3
    dZ3_in = Y - Z3
    dZ3 = dZ3_in * sigmoid_deriv(Z3)
    # layer 2
    dZ2_in = dZ3.dot(W3.T)
    dZ2 = dZ2_in * relu_deriv(Z2)
    # layer 1
    dZ1_in = dZ2.dot(W2.T)
    dZ1 = dZ1_in * relu_deriv(Z1)
    # changes in weights
    dW1 = X.T.dot(dZ1) / m
    dW2 = Z1.T.dot(dZ2) / m
    dW3 = Z2.T.dot(dZ3) / m
    # chnages in biases
    dB1 = np.sum(dZ1) / m 
    dB2 = np.sum(dZ2) / m
    dB3 = np.sum(dZ3) / m
    return dW1, dW2, dW3, dB1, dB2, dB3

def update_param(W1, W2, W3, dW1, dW2, dW3, B1, B2, B3, dB1, dB2, dB3, A, M):
    # update weights
    W1 += dW1 * A + dW1 * M
    W2 += dW2 * A + dW2 * M
    W3 += dW3 * A + dW3 * M
    # update biases
    B1 += dB1 * A + dB1 * M
    B2 += dB2 * A + dB2 * M
    B3 += dB3 * A + dB3 * M
    return W1, W2, W3, B1, B2, B3

def __MAIN__(X_train, Y_train, A, M, T, epoch):
    X, Y, W1, W2, W3, B1, B2, B3 = init_param(X_train, Y_train)
    acc_list = []
    epoch_range = []
    mse_list = []
    # bpnn
    for i in range(epoch):
        Z1, Z2, Z3 = feedforward(W1, W2, W3, B1, B2, B3, X)
        dW1, dW2, dW3, dB1, dB2, dB3 = backprop(Z1, Z2, Z3, W1, W2, W3, X, Y)
        W1, W2, W3, B1, B2, B3 = update_param(W1, W2, W3, dW1, dW2, dW3, B1, B2, B3, dB1, dB2, dB3, A, M)
        
        mse = mean_squared_error(Y_train, Z3)
        y_pred = get_predictions(Z3, T)
        acc = get_accuracy(y_pred, Y_train)
        
        acc_list.append(float(acc))
        epoch_range.append(float(i))
        mse_list.append(float(mse))
        if (i % 1 == 0):
            print("Epoch:", i)
            print("Accuracy\t   :", get_accuracy(y_pred, Y_train))
            print("Loss\t\t   :", 100 - get_accuracy(y_pred, Y_train))
            print("Mean Squared Error :", mse)
            print("---------------------------------------")
    plt.plot(epoch_range, acc_list)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.show()

    plt.plot(epoch_range, mse_list)
    plt.ylabel('mean squared error')
    plt.xlabel('epoch')
    plt.show()
    return W1, W2, W3, B1, B2, B3

In [ ]:
learning_rate = 0.05
momentum = 0.8
threshold = 0.5
epoch = 300

W1, W2, W3, B1, B2, B3 = __MAIN__(X_train, Y_train, learning_rate, momentum, threshold, epoch)

In [ ]:
def test_prediction(W1, W2, W3, B1, B2, B3, X_test):
    _, _, Z3 = feedforward(W1, W2, W3, B1, B2, B3, np.array(X_test))
    mse = mean_squared_error(Y_test, Z3)
    y_pred = get_predictions(Z3, 0.5)
    
    print("Accuracy\t:", get_accuracy(y_pred, Y_test))
    print("Loss\t\t   :", 100 - get_accuracy(y_pred, Y_test))
    print("Mean Squared Error :", mse)

    pred_out = []
    act_out = []
    loss_one = 0
    loss_zero = 0
    counter = 0
    for i in y_pred:
        if Y_test[counter] == 1:
            if i !=  Y_test[[counter] ]:
                loss_one += 1
        else:
            if i !=  Y_test[counter]:
                loss_zero += 1
        pred_out.append(i)
        counter += 1
    for i in Y_test:
        act_out.append(i)
    tested_data = {'Predicted Output':pred_out, 'Actual Output':act_out}
    return pd.DataFrame(tested_data), loss_one, loss_zero

tested_data, loss_one, loss_zero = test_prediction(W1, W2, W3, B1, B2, B3, X_test)

tested_data

In [ ]:
#Grid plot set up
fig = plt.figure(figsize=(10,10)) 
fig_dims = (3, 2)

#Plotting for Predicted and Actual Output
plt.subplot2grid(fig_dims, (0, 0))
tested_data['Predicted Output'].value_counts().plot(kind='bar', 
                                     title='Predicted Output')
plt.subplot2grid(fig_dims, (0, 1))
tested_data['Actual Output'].value_counts().plot(kind='bar', 
                                     title='Actual Output')

#ested_data
tested_data.apply(pd.value_counts).plot(kind='bar', title='all types')

print("Unmatched Ones:", loss_one, "\nUnmatched Zeros:", loss_zero)